In [5]:
#####Random Search 
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(df_train[features], df_train["Churn"])

rf_random.best_params_

# 9. Evaluate model pipeline on test data
predrf = rf_random.predict(df_test[features])
print("Accuracy: " ,metrics.accuracy_score(df_test["Churn"], predrf))
print(metrics.classification_report(df_test["Churn"], predrf))
print(metrics.confusion_matrix(df_test["Churn"], predrf))


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


NameError: name 'rf' is not defined

In [6]:
import pandas as pd
df = pd.read_csv('Camera.csv', sep=';')

columns = ['Model', 'Date', 'MaxRes', 'LowRes', 'EffPix', 'ZoomW', 'ZoomT',
         'NormalFR', 'MacroFR', 'Storage', 'Weight', 'Dimensions', 'Price']
df.columns = columns
df.head()

df.describe()

df['Model'] = df['Model'].apply(lambda s:s.split()[0])
df['Model']=df['Model'].astype('object')
df.dtypes

df1=df.drop(["Model", "Date"], axis=1)
df2=df1.dropna()
df.shape

## Dimension Reduction 

from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(df2)
print("explained_variance")
print(pca.explained_variance_ratio_)
print("Components")
print(pca.components_)

import pandas as pd
import pylab as plt
import numpy as np
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans


from sklearn.metrics import silhouette_samples, silhouette_score

range_n_clusters = [2, 3, 4, 5, 6,7,8,9,10]

# Compute the silhouette scores for each sample
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(df2)
    #sample_silhouette_values = silhouette_samples(X, cluster_labels)
    #print("%d Cluster size %f ",n_clusters,sample_silhouette_values)
    
    silhouette_avg = silhouette_score(df2, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)
from sklearn import metrics
k = range(1,11)

clusters = [KMeans(n_clusters = c,init = 'k-means++').fit(df2) 
            for c in k]
centr_lst = [cc.cluster_centers_ for cc in clusters]

k_distance = [cdist(df2, cent, 'euclidean') for cent in centr_lst]
clust_indx = [np.argmin(kd,axis=1) for kd in k_distance]
distances = [np.min(kd,axis=1) for kd in k_distance]
avg_within = [np.sum(dist)/df2.shape[0] for dist in distances]

kidx = 2

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(k, avg_within, 'g*-')
ax.plot(k[kidx], avg_within[kidx], marker='o', markersize=12, \
markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elbow for KMeans clustering (Camera  Data)')

from sklearn import cluster
k_means = cluster.KMeans(n_clusters=3)
kmf=k_means.fit(df2)

P_kmeans = kmf.predict(df2)
#P_kmeans=P_kmeans.astype('object')
Clust=pd.DataFrame(P_kmeans)

dfwithClust=pd.concat([df,Clust], axis=1)
dfwithClust.rename(columns={0:'Clust'},inplace=True)
dfwithClust.head()

dfwithClust.Clust.value_counts()

explained_variance
[0.62424922 0.29280832 0.05466678 0.02330201 0.00416335]
Components
[[ 6.53737141e-01  7.20858960e-01  2.36333946e-03 -3.38883346e-03
   7.34742049e-03 -4.52378280e-03 -2.59893216e-03  3.59907825e-03
   2.46009613e-02 -2.60557259e-04  2.28634224e-01]
 [-1.25384784e-01 -1.95264459e-01 -4.13315656e-04 -5.48171673e-03
  -2.91444145e-02 -7.67014840e-03 -2.61885218e-04 -6.03789168e-03
   1.76583559e-01  1.03958241e-02  9.55973665e-01]
 [ 7.04994558e-01 -6.19856431e-01  2.20394828e-03 -1.19866987e-02
  -3.33594803e-02 -1.32007015e-02 -2.48403578e-03  5.82210663e-04
   3.28028313e-01  2.16656939e-02 -9.61589310e-02]
 [-2.42645194e-01  2.36377460e-01 -1.12356454e-03 -1.96945538e-02
   6.07941237e-02 -2.72625668e-02 -6.25447800e-03 -1.55448187e-02
   9.23524304e-01  6.09832557e-02 -1.53374829e-01]
 [ 2.99860410e-02 -4.61356135e-02  7.05369054e-04  3.86308424e-02
   9.94901129e-01  4.30997208e-02 -3.45596981e-03  2.88259643e-02
  -3.62555122e-02 -2.40203465e-02  3.25475246e-02

1.0    550
0.0    471
2.0     15
Name: Clust, dtype: int64